In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data=pd.read_excel('/content/FullT20wc2024.xlsx')

In [ ]:
data.head()

,match_id,custom_match_id,season,type_clean,competition_code,competition,gender,venue,date,timestamp,...,penalties,extras,out_player_cricinfo_id,out_player_id,out_player,dismissal_type,dismissal_text_shot,dismissal_text_long,dismissal_text_commentary,Stage
0,1415701,ICCMTWC_USA_CAN_2024-06-01,2024,T20,ICCMTWC,ICC Men's T20 World Cup,male,"Grand Prairie Stadium, Dallas",2024-06-01,2024-06-02T00:37:08.000Z,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Group Stage
1,1415701,ICCMTWC_USA_CAN_2024-06-01,2024,T20,ICCMTWC,ICC Men's T20 World Cup,male,"Grand Prairie Stadium, Dallas",2024-06-01,2024-06-02T00:37:39.000Z,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Group Stage
2,1415701,ICCMTWC_USA_CAN_2024-06-01,2024,T20,ICCMTWC,ICC Men's T20 World Cup,male,"Grand Prairie Stadium, Dallas",2024-06-01,2024-06-02T00:38:17.000Z,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Group Stage
3,1415701,ICCMTWC_USA_CAN_2024-06-01,2024,T20,ICCMTWC,ICC Men's T20 World Cup,male,"Grand Prairie Stadium, Dallas",2024-06-01,2024-06-02T00:38:54.000Z,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Group Stage
4,1415701,ICCMTWC_USA_CAN_2024-06-01,2024,T20,ICCMTWC,ICC Men's T20 World Cup,male,"Grand Prairie Stadium, Dallas",2024-06-01,2024-06-02T00:39:29.000Z,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Group Stage


In [ ]:
data.shape

(11381, 56)

In [ ]:
data.columns

Index(['match_id', 'custom_match_id', 'season', 'type_clean',
       'competition_code', 'competition', 'gender', 'venue', 'date',
       'timestamp', 'inning_number', 'id', 'overs_unique', 'overs_actual',
       'over_number', 'ball_number', 'balls_remaining', 'total_inning_runs',
       'run_rate', 'target', 'total_inning_wickets', 'bowl_team',
       'bowl_team_id', 'bowl_team_abbr', 'bowler_cricinfo_id',
       'bowler_player_id', 'bowler', 'bowling_hand', 'bowling_pacespin',
       'bowling_style_long', 'bowler_wickets_cumulative', 'title', 'bat_team',
       'bat_team_id', 'bat_team_abbr', 'batter_cricinfo_id',
       'batter_player_id', 'batter', 'batting_style', 'batter_runs_cumulative',
       'total_runs', 'runs_off_bat', 'byes', 'legbyes', 'wides', 'noballs',
       'penalties', 'extras', 'out_player_cricinfo_id', 'out_player_id',
       'out_player', 'dismissal_type', 'dismissal_text_shot',
       'dismissal_text_long', 'dismissal_text_commentary', 'Stage'],
      dtype='ob

In [ ]:
#OVERALL METRICS

#highest runs

# Calculate total runs per batter
total_runs_per_batter = data.groupby('batter')['total_runs'].sum().reset_index(name='Total_Runs_Tournament')

# Calculate total balls faced per batter
total_balls_per_batter = data[(data['wides'] == 0) & (data['noballs'] == 0)].groupby('batter')['overs_actual'].count().reset_index(name='total_balls_used')

# Calculate highest score per batter using batter_runs_cumulative
highest_score_per_batter = data.groupby('batter')['batter_runs_cumulative'].max().reset_index(name='highest_score')

# Merge the total runs, total balls, and highest score data
batter_stats = pd.merge(total_runs_per_batter, total_balls_per_batter, on='batter')
batter_stats = pd.merge(batter_stats, highest_score_per_batter, on='batter')

# Get the team information
bat_team_info = data[['batter', 'bat_team']].drop_duplicates()

# Merge with the batter stats
batter_stats = pd.merge(batter_stats, bat_team_info, on='batter')

# Sort by total runs in descending order and add rank
batter_stats = batter_stats.sort_values(by='Total_Runs_Tournament', ascending=False)
batter_stats['rank'] = range(1, len(batter_stats) + 1)

# Filter the top 5 run-scorers
top_5_run_scorers = batter_stats.head(5)

# Arrange columns as specified
top_5_run_scorers = top_5_run_scorers[['batter', 'bat_team', 'Total_Runs_Tournament', 'total_balls_used', 'rank', 'highest_score']]

# Display the final DataFrame
print(top_5_run_scorers)


                 batter      bat_team  Total_Runs_Tournament  \
179  Rahmanullah Gurbaz   Afghanistan                    303   
234         Travis Head     Australia                    265   
192        Rohit Sharma         India                    264   
176     Quinton de Kock  South Africa                    256   
82       Ibrahim Zadran   Afghanistan                    255   

     total_balls_used  rank  highest_score  
179               226     1             80  
234               161     2             76  
192               164     3             92  
176               170     4             74  
82                214     5             70  


In [ ]:
#Highest 4s
# Filter data for runs that are 4
fours_data = data[data['runs_off_bat'] == 4]

# Group by 'batter' and 'bat_team' to calculate the count of 4s and the total runs from 4s
four_stats = fours_data.groupby(['batter', 'bat_team']).agg(four_count=('runs_off_bat', 'size'), four_runs=('runs_off_bat', 'sum')).reset_index()

# Sort by 'four_count' in descending order and select the top 5
top_5_fours = four_stats.sort_values(by='four_count', ascending=False).head(5)

# Add rank
top_5_fours['rank'] = range(1, len(top_5_fours) + 1)

# Display the final DataFrame
print(top_5_fours)



              batter      bat_team  four_count  four_runs  rank
168      Travis Head     Australia          26        104     1
61    Ibrahim Zadran   Afghanistan          25        100     2
138     Rohit Sharma         India          24         96     3
71       Jos Buttler       England          22         88     4
127  Quinton de Kock  South Africa          21         84     5


In [ ]:
#Highest 6s
# Filter data for runs that are 6
six_data = data[data['runs_off_bat'] == 6]

# Group by 'batter' and 'bat_team' to calculate the count of 6s and the total runs from 6s
six_stats = six_data.groupby(['batter', 'bat_team']).agg(six_count=('runs_off_bat', 'size'), six_runs=('runs_off_bat', 'sum')).reset_index()

# Sort by 'six_count' in descending order and select the top 5
top_5_six = six_stats.sort_values(by='six_count', ascending=False).head(5)

# Add rank
top_5_six['rank'] = range(1, len(top_5_six) + 1)

# Display the final DataFrame
print(top_5_six)

                 batter                  bat_team  six_count  six_runs  rank
89      Nicholas Pooran               West Indies         17       102     1
99   Rahmanullah Gurbaz               Afghanistan         16        96     2
107        Rohit Sharma                     India         15        90     3
129         Travis Head                 Australia         15        90     4
1           Aaron Jones  United States of America         14        84     5


In [ ]:
#Highest Wickets


# Step 1: Filter rows where wickets were taken (out_player_id is not null) and the dismissal is not "run out"
wickets_data = data[(data['out_player_id'].notnull()) & (data['dismissal_text_shot'] != 'run out')]

# Step 2: Group by bowler_player_id and count occurrences of out_player_id
wickets_count = wickets_data.groupby('bowler_player_id')['out_player_id'].count().reset_index(name='Total_wickets_WC')

# Step 3: Sort the wickets_count in descending order and select top 5
top_5_wicket_takers = wickets_count.nlargest(5, 'Total_wickets_WC')

# Step 4: Retrieve names and bowl teams for the top 5 wicket takers
top_5_wkts_info = data[data['bowler_player_id'].isin(top_5_wicket_takers['bowler_player_id'])][['bowler_player_id', 'bowler', 'bowl_team']]
top_5_wkts_info = top_5_wkts_info.drop_duplicates().reset_index(drop=True)

# Step 5: Merge with wickets_count to get total wickets and sort by it
top_5_wkts_info = top_5_wkts_info.merge(top_5_wicket_takers, on='bowler_player_id')
top_5_wkts_info= top_5_wkts_info.sort_values(by='Total_wickets_WC', ascending=False).reset_index(drop=True)

# Step 6: Add Rank column
top_5_wkts_info['Rank'] = top_5_wkts_info['Total_wickets_WC'].rank(method='min', ascending=False).astype(int)

# Display the sorted top 5 wicket takers along with their names, bowl teams, total wickets, and rank
print("Top 5 Wicket Takers:")
print(top_5_wkts_info[['bowler', 'bowl_team', 'Total_wickets_WC', 'Rank']])


Top 5 Wicket Takers:
             bowler     bowl_team  Total_wickets_WC  Rank
0  Fazalhaq Farooqi   Afghanistan                17     1
1    Arshdeep Singh         India                17     1
2     Anrich Nortje  South Africa                15     3
3    Jasprit Bumrah         India                15     3
4       Rashid Khan   Afghanistan                14     5


In [ ]:
#Lowest economy

# Step 1: Remove duplicate over_number values for each bowler within each match
unique_overs = data[['match_id', 'bowler', 'bowl_team', 'over_number']].drop_duplicates()

# Step 2: Group by bowler to count the unique overs bowled by each bowler
bowler_total_overs = unique_overs.groupby('bowler')['over_number'].count().reset_index(name='total_overs')

# Step 3: Group by bowler to sum the total runs conceded by each bowler
bowler_total_runs_conceded = data.groupby('bowler')['total_runs'].sum().reset_index(name='total_runs_conceded')

# Step 4: Calculate the economy rate for each bowler
economy_rate_df = pd.merge(bowler_total_overs, bowler_total_runs_conceded, on='bowler')
economy_rate_df['economy_rate'] = economy_rate_df['total_runs_conceded'] / economy_rate_df['total_overs']

# Step 5: Merge bowl_team into economy_rate_df
economy_rate_df = pd.merge(economy_rate_df, data[['bowler', 'bowl_team']].drop_duplicates(), on='bowler', how='left')

# Step 6: Filter bowlers who have bowled 30 or more overs
qualified_bowlers = economy_rate_df[economy_rate_df['total_overs'] >= 30]

# Step 7: Get top 5 bowlers with lowest economy rates
top_5_bowlers_eco = qualified_bowlers.nsmallest(5, 'economy_rate')

# Step 8: Add Rank column
top_5_bowlers_eco['Rank'] = range(1, len(top_5_bowlers_eco) + 1)

# Step 9: Display the top 5 bowlers with lowest economy rates
print("\nTop 5 Bowlers with Lowest Economy Rates (30 or More Overs):")
print(top_5_bowlers_eco[['bowler', 'bowl_team', 'economy_rate', 'total_overs', 'total_runs_conceded', 'Rank']])







Top 5 Bowlers with Lowest Economy Rates (30 or More Overs):
            bowler     bowl_team  economy_rate  total_overs  \
67  Jasprit Bumrah         India      4.600000           30   
12   Anrich Nortje  South Africa      5.971429           35   
76   Kagiso Rabada  South Africa      6.451613           31   
91    Marco Jansen  South Africa      6.875000           32   
14  Arshdeep Singh         India      7.433333           30   

    total_runs_conceded  Rank  
67                  138     1  
12                  209     2  
76                  200     3  
91                  220     4  
14                  223     5  


In [ ]:
#Wides count
# Filter rows where wides were bowled
wides_data = data[data['wides'] > 0]

# Group by the bowling team to get the sum of wides runs for each team
wides_runs_by_team = wides_data.groupby('bowl_team')['wides'].sum().reset_index(name='wides_runs')

# Group by the bowling team to get the count of wides for each team
wides_count_by_team = wides_data.groupby('bowl_team')['wides'].count().reset_index(name='wides_count')

# Merge the two results on 'bowl_team'
wides_stats_by_team = pd.merge(wides_runs_by_team, wides_count_by_team, on='bowl_team')

# Sort the results by wides_count in descending order
wides_stats_by_team = wides_stats_by_team.sort_values(by='wides_count', ascending=False)

# Add Rank column
wides_stats_by_team['Rank'] = range(1, len(wides_stats_by_team) + 1)

# Display the result
print("Most wides by teams")
print(wides_stats_by_team[['bowl_team', 'wides_runs', 'wides_count', 'Rank']])


Most wides by teams
                   bowl_team  wides_runs  wides_count  Rank
15              South Africa          46           46     1
1                  Australia          41           35     2
0                Afghanistan          54           33     3
4                    England          30           30     4
19               West Indies          37           30     5
5                      India          36           27     6
12                  Pakistan          30           22     7
3                     Canada          28           20     8
18  United States of America          25           20     9
16                 Sri Lanka          19           19    10
9                Netherlands          17           17    11
2                 Bangladesh          28           17    12
13          Papua New Guinea          20           15    13
6                    Ireland          14           14    14
17                    Uganda          17           12    15
10               New

In [ ]:
#Most dot balls by bat team
# Filter rows where no runs were scored (dot balls)
dots = data[data['total_runs'] == 0]

# Group by the batting team to get the count of dot balls for each team
dots_p = dots.groupby('bat_team')['total_runs'].count().reset_index(name='total_dot_balls')

# Sort the results by total_dot_balls in descending order
dots_by_team = dots_p.sort_values(by='total_dot_balls', ascending=False)

# Add Rank column
dots_by_team['Rank'] = range(1, len(dots_by_team) + 1)

# Display the result
print("Most Dot Balls Played by a Batting Team:")
print(dots_by_team[['bat_team', 'total_dot_balls', 'Rank']])

Most Dot Balls Played by a Batting Team:
                    bat_team  total_dot_balls  Rank
15              South Africa              420     1
0                Afghanistan              397     2
2                 Bangladesh              364     3
5                      India              340     4
19               West Indies              315     5
18  United States of America              305     6
1                  Australia              269     7
13          Papua New Guinea              265     8
17                    Uganda              262     9
11                      Oman              212    10
9                Netherlands              211    11
8                      Nepal              207    12
7                    Namibia              197    13
12                  Pakistan              196    14
4                    England              190    15
6                    Ireland              182    16
16                 Sri Lanka              160    17
3                     C

In [ ]:
# Filter rows where no runs were scored (dot balls)
dots = data[data['total_runs'] == 0]

# Group by batter and batting team to get the count of dot balls for each batter
dots_p = dots.groupby(['batter', 'bat_team'])['total_runs'].count().reset_index(name='total_dot_balls')

# Get the top 5 batters with the most dot balls
most_dot_balls_batters = dots_p.nlargest(5, 'total_dot_balls')

# Add Rank column
most_dot_balls_batters['Rank'] = range(1, len(most_dot_balls_batters) + 1)

# Display the result
print("Most Dot Balls Played by a Batter:")
print(most_dot_balls_batters[['batter', 'bat_team', 'total_dot_balls', 'Rank']])


Most Dot Balls Played by a Batter:
                 batter      bat_team  total_dot_balls  Rank
176  Rahmanullah Gurbaz   Afghanistan               99     1
80       Ibrahim Zadran   Afghanistan               95     2
48         David Miller  South Africa               76     3
181     Riazat Ali Shah        Uganda               72     4
118          Litton Das    Bangladesh               69     5


In [ ]:
# Filter data for boundaries and exclude 'Group Stage'
stage = data[(data['runs_off_bat'].isin([4, 6])) & (data['Stage'] != 'Group Stage')]

# Group by 'bat_team' to count the number of boundary runs (4s and 6s)
boundary_sum_per_team = stage.groupby('bat_team')['runs_off_bat'].sum().reset_index(name='total_boundary_runs')

# Calculate total runs per team
total_runs_per_team = data[data['Stage'] != 'Group Stage'].groupby('bat_team')['total_runs'].sum().reset_index(name='total_runs')

# Merge the boundary runs and total runs data
boundary_rate_df = pd.merge(boundary_sum_per_team, total_runs_per_team, on='bat_team')

# Calculate the boundary rate as a percentage
boundary_rate_df['boundary_rate'] = (boundary_rate_df['total_boundary_runs'] / boundary_rate_df['total_runs']) * 100

# Sort the DataFrame by boundary rate in descending order
boundary_rate_df = boundary_rate_df.sort_values(by='boundary_rate', ascending=False)

# Add rank column
boundary_rate_df['rank'] = boundary_rate_df['boundary_rate'].rank(method='min', ascending=False).astype(int)

# Display the DataFrame with boundary rates and rank
print("\nBoundary Rate for Teams (Top-8, Excluding Group Stage):")
print(boundary_rate_df)



Boundary Rate for Teams (Top-8, Excluding Group Stage):
                   bat_team  total_boundary_runs  total_runs  boundary_rate  \
7               West Indies                  280         445      62.921348   
4                     India                  572         929      61.571582   
1                 Australia                  248         408      60.784314   
5              South Africa                  402         710      56.619718   
6  United States of America                  226         419      53.937947   
3                   England                  298         557      53.500898   
0               Afghanistan                  224         453      49.448124   
2                Bangladesh                  192         391      49.104859   

   rank  
7     1  
4     2  
1     3  
5     4  
6     5  
3     6  
0     7  
2     8  


In [ ]:
# Filter data for boundaries and exclude 'Group Stage'
stage = data[(data['runs_off_bat'].isin([4, 6])) & (data['Stage'] != 'Group Stage')]

# Group by 'batter' to count the number of boundary runs (4s and 6s)
boundary_sum_per_batter = stage.groupby('batter')['runs_off_bat'].sum().reset_index(name='total_boundary_runs')

# Calculate total runs per batter
total_runs_per_batter = data[data['Stage'] != 'Group Stage'].groupby('batter')['total_runs'].sum().reset_index(name='total_runs')

# Calculate total balls faced per batter
total_balls_per_batter = data[(data['Stage'] != 'Group Stage') & (data['wides'] == 0) & (data['noballs'] == 0)].groupby('batter')['overs_actual'].count().reset_index(name='total_balls')

# Get the unique batters and their corresponding teams
unique_batters_teams = data[data['Stage'] != 'Group Stage'][['batter', 'bat_team']].drop_duplicates()

# Merge the boundary runs, total runs, and total balls data
boundary_rate_batter_df = pd.merge(boundary_sum_per_batter, total_runs_per_batter, on='batter')
boundary_rate_batter_df = pd.merge(boundary_rate_batter_df, total_balls_per_batter, on='batter')
boundary_rate_batter_df = pd.merge(boundary_rate_batter_df, unique_batters_teams, on='batter')

# Calculate the boundary rate as a percentage
boundary_rate_batter_df['boundary_rate'] = (boundary_rate_batter_df['total_boundary_runs'] / boundary_rate_batter_df['total_runs']) * 100

# Filter batters who have faced 50 or more balls
boundary_rate_batter_df = boundary_rate_batter_df[boundary_rate_batter_df['total_balls'] >= 50]

# Sort the DataFrame by boundary rate in descending order and keep only the top 10
boundary_rate_batter_df = boundary_rate_batter_df.sort_values(by='boundary_rate', ascending=False).head(10)

# Add rank column
boundary_rate_batter_df['rank'] = boundary_rate_batter_df['boundary_rate'].rank(method='min', ascending=False).astype(int)

# Display the DataFrame with boundary rates
print(boundary_rate_batter_df[['batter', 'bat_team', 'total_boundary_runs', 'total_runs', 'total_balls', 'boundary_rate', 'rank']])


              batter                  bat_team  total_boundary_runs  \
72       Travis Head                 Australia                   84   
55      Rohit Sharma                     India                  142   
15     Glenn Maxwell                 Australia                   66   
67  Suryakumar Yadav                     India                   96   
48   Quinton de Kock              South Africa                  136   
53      Rishabh Pant                     India                   54   
18     Hardik Pandya                     India                   94   
7       Andries Gous  United States of America                   74   
26       Jos Buttler                   England                   94   
21  Heinrich Klaasen              South Africa                   78   

    total_runs  total_balls  boundary_rate  rank  
72         114           67      73.684211     1  
55         193          109      73.575130     2  
15          95           59      69.473684     3  
67         142

In [ ]:
#BOUNDARY RATE
#BOWLING

# Filter data for boundaries (4s and 6s)
boundary_data = data[data['runs_off_bat'].isin([4, 6])]

# Group by 'bowler' to count boundaries conceded and total balls bowled
boundary_count_per_bowler = boundary_data.groupby('bowler')['runs_off_bat'].count().reset_index(name='boundary_count')
balls_bowled_per_bowler = data.groupby('bowler')['overs_actual'].count().reset_index(name='balls_bowled')

# Get the unique bowlers and their corresponding teams
unique_bowlers_teams = data[['bowler', 'bowl_team']].drop_duplicates()

# Merge boundary count, balls bowled data, and bowl team
boundary_frequency_df = pd.merge(boundary_count_per_bowler, balls_bowled_per_bowler, on='bowler')
boundary_frequency_df = pd.merge(boundary_frequency_df, unique_bowlers_teams, on='bowler')

# Calculate boundary frequency rate (boundaries conceded per ball)
boundary_frequency_df['boundary_rate_bowler'] = boundary_frequency_df['balls_bowled'] / boundary_frequency_df['boundary_count']

# Filter bowlers who have bowled at least 15 overs (assuming 1 over = 6 balls)
boundary_frequency_df = boundary_frequency_df[boundary_frequency_df['balls_bowled'] >= 120]  # 20 overs * 6 balls per over

# Sort the DataFrame by boundary frequency rate in ascending order and keep only the top 5
top_5_bowlers_boundary_frequency = boundary_frequency_df.sort_values(by='boundary_rate_bowler', ascending=True).head(5)

# Add rank column
top_5_bowlers_boundary_frequency['rank'] = top_5_bowlers_boundary_frequency['boundary_rate_bowler'].rank(method='min', ascending=True).astype(int)

# Display the top 5 bowlers with lowest boundary frequency rates
print("Top 5 Bowlers with Lowest Boundary Count:")
print(top_5_bowlers_boundary_frequency[['bowler', 'bowl_team', 'boundary_rate_bowler', 'rank']])






Top 5 Bowlers with Lowest Boundary Count:
             bowler                 bowl_team  boundary_rate_bowler  rank
68     Jofra Archer                   England              6.040000     1
7          Ali Khan  United States of America              6.047619     2
152    Taskin Ahmed                Bangladesh              6.190476     3
56    Hardik Pandya                     India              6.280000     4
128  Rishad Hossain                Bangladesh              6.695652     5


In [ ]:
#BATTING
#AVG RUN RATE(PP)
# Define a function to calculate average run rate during powerplay
def calculate_avg_run_rate(data, stage):
    filtered_data = data[(data['Stage'] == stage) & (data['over_number'] >= 1) & (data['over_number'] <= 6)]
    team_avg = filtered_data.groupby('bat_team')['run_rate'].mean()
    return team_avg.reset_index(name=f'{stage}_avg_runrate_PP')

# Calculate average run rates for each stage
group_stage_avg = calculate_avg_run_rate(data, 'Group Stage')
top_8_avg = calculate_avg_run_rate(data, 'Super_8')
semi_final_avg = calculate_avg_run_rate(data, 'Semi_final')
final_avg = calculate_avg_run_rate(data, 'Final')

# Merge the results into a single DataFrame
result_df = group_stage_avg.merge(top_8_avg, on='bat_team', how='left')
result_df = result_df.merge(semi_final_avg, on='bat_team', how='left')
result_df = result_df.merge(final_avg, on='bat_team', how='left')

# Rename columns for clarity
result_df.columns = [
    'bat_team',
    'Group_stage_avg_runrate_PP',
    'Top_8_avg_runrate_PP',
    'Semi_final_avg_runrate_PP',
    'Final_avg_runrate_PP'
]

# Print the final DataFrame
print(result_df)


                    bat_team  Group_stage_avg_runrate_PP  \
0                Afghanistan                    7.010000   
1                  Australia                    8.368333   
2                 Bangladesh                    4.723113   
3                     Canada                    8.071875   
4                    England                    9.374947   
5                      India                    6.560522   
6                    Ireland                    4.615508   
7                    Namibia                    5.028600   
8                      Nepal                    4.814182   
9                Netherlands                    5.211267   
10               New Zealand                    5.429276   
11                      Oman                    5.419128   
12                  Pakistan                    6.108543   
13          Papua New Guinea                    4.055789   
14                  Scotland                    5.526053   
15              South Africa            

In [ ]:
#AVG RUN RATE(mo)

# Define a function to calculate average run rate during middle overs
def calculate_avg_run_rate_middle_overs(data, stage):
    filtered_data = data[(data['Stage'] == stage) & (data['over_number'] >= 7) & (data['over_number'] <= 15)]
    team_avg = filtered_data.groupby('bat_team')['run_rate'].mean()
    return team_avg.reset_index(name=f'{stage}_avg_runrate_MO')

# Calculate average run rates for each stage during middle overs
group_stage_avg_mo = calculate_avg_run_rate_middle_overs(data, 'Group Stage')
top_8_avg_mo = calculate_avg_run_rate_middle_overs(data, 'Super_8')
semi_final_avg_mo = calculate_avg_run_rate_middle_overs(data, 'Semi_final')
final_avg_mo = calculate_avg_run_rate_middle_overs(data, 'Final')

# Merge the results into a single DataFrame
result_df_mo = group_stage_avg_mo.merge(top_8_avg_mo, on='bat_team', how='left')
result_df_mo = result_df_mo.merge(semi_final_avg_mo, on='bat_team', how='left')
result_df_mo = result_df_mo.merge(final_avg_mo, on='bat_team', how='left')

# Rename columns for clarity
result_df_mo.columns = [
    'bat_team',
    'Group_stage_avg_runrate_MO',
    'Top_8_avg_runrate_MO',
    'Semi_final_avg_runrate_MO',
    'Final_avg_runrate_MO'
]

# Print the final DataFrame for middle overs
print(result_df_mo)


                    bat_team  Group_stage_avg_runrate_MO  \
0                Afghanistan                    7.616438   
1                  Australia                    7.876190   
2                 Bangladesh                    6.240308   
3                     Canada                    6.766946   
4                    England                    9.515443   
5                      India                    6.435461   
6                    Ireland                    5.175879   
7                    Namibia                    5.575632   
8                      Nepal                    4.909647   
9                Netherlands                    5.920586   
10               New Zealand                    5.537315   
11                      Oman                    5.520829   
12                  Pakistan                    5.880864   
13          Papua New Guinea                    4.539819   
14                  Scotland                    8.974679   
15              South Africa            

In [ ]:
#AVG RUN RATE(DEATH)

# Define a function to calculate average run rate during death overs
def calculate_avg_run_rate_death_overs(data, stage):
    filtered_data = data[(data['Stage'] == stage) & (data['over_number'] >= 16) & (data['over_number'] <= 20)]
    team_avg = filtered_data.groupby('bat_team')['run_rate'].mean()
    return team_avg.reset_index(name=f'{stage}_avg_runrate_DO')

# Calculate average run rates for each stage during death overs
group_stage_avg_do = calculate_avg_run_rate_death_overs(data, 'Group Stage')
top_8_avg_do = calculate_avg_run_rate_death_overs(data, 'Super_8')
semi_final_avg_do = calculate_avg_run_rate_death_overs(data, 'Semi_final')
final_avg_do = calculate_avg_run_rate_death_overs(data, 'Final')

# Merge the results into a single DataFrame
result_df_do = group_stage_avg_do.merge(top_8_avg_do, on='bat_team', how='left')
result_df_do = result_df_do.merge(semi_final_avg_do, on='bat_team', how='left')
result_df_do = result_df_do.merge(final_avg_do, on='bat_team', how='left')

# Rename columns for clarity
result_df_do.columns = [
    'bat_team',
    'Group_stage_avg_runrate_DO',
    'Top_8_avg_runrate_DO',
    'Semi_final_avg_runrate_DO',
    'Final_avg_runrate_DO'
]

# Print the final DataFrame for death overs
print(result_df_do)


                    bat_team  Group_stage_avg_runrate_DO  \
0                Afghanistan                    8.380000   
1                  Australia                    8.949792   
2                 Bangladesh                    6.118235   
3                     Canada                    7.084043   
4                    England                    8.130000   
5                      India                    5.940667   
6                    Ireland                    5.675373   
7                    Namibia                    6.122973   
8                      Nepal                    5.279157   
9                Netherlands                    5.855789   
10               New Zealand                    5.940625   
11                      Oman                    6.189889   
12                  Pakistan                    6.248155   
13          Papua New Guinea                    4.839655   
14                  Scotland                    8.592000   
15              South Africa            

In [ ]:
#highest SR pp

# Filter data for group stage and powerplay overs (1-6)
group_stage_powerplay_data = data[(data['Stage'] == 'Group Stage') & (data['over_number'] >= 1) & (data['over_number'] <= 6)]

# Remove duplicate over_number values for each batter within each match in powerplay
unique_powerplay_batters = group_stage_powerplay_data[['match_id', 'batter', 'bat_team']].drop_duplicates()

# Group by batter to calculate total runs scored by each batter in powerplay
batter_total_powerplay_runs = group_stage_powerplay_data.groupby('batter')['total_runs'].sum().reset_index(name='total_powerplay_runs')

# Calculate balls faced by each batter in powerplay
batter_balls_faced_powerplay = group_stage_powerplay_data.groupby('batter')['overs_actual'].count().reset_index(name='balls_faced_powerplay')

# Merge total runs and balls faced data
batter_powerplay_stats = pd.merge(batter_total_powerplay_runs, batter_balls_faced_powerplay, on='batter')

# Calculate strike rate for each batter in powerplay
batter_powerplay_stats['strike_rate_powerplay'] = (batter_powerplay_stats['total_powerplay_runs'] / batter_powerplay_stats['balls_faced_powerplay']) * 100

# Merge with 'bat_team' to get team information
batter_powerplay_stats = pd.merge(batter_powerplay_stats, unique_powerplay_batters[['batter', 'bat_team']], on='batter')

# Filter batters with 20 or more balls faced in powerplay
batter_powerplay_stats = batter_powerplay_stats[batter_powerplay_stats['balls_faced_powerplay'] >= 50]

# Sort by strike rate in descending order and drop duplicates to keep unique batters
unique_batters_sorted = batter_powerplay_stats.sort_values(by='strike_rate_powerplay', ascending=False).drop_duplicates(subset=['batter'])

# Get top 5 unique batters with highest strike rates in powerplay
top_5_powerplay_batters = unique_batters_sorted.head(5)

# Add rank column
top_5_powerplay_batters['rank'] = top_5_powerplay_batters['strike_rate_powerplay'].rank(method='min', ascending=False).astype(int)

# Save the top 5 batters with highest strike rates in powerplay to a DataFrame
top_5_powerplay_batters_df = top_5_powerplay_batters[['batter', 'bat_team', 'total_powerplay_runs', 'strike_rate_powerplay', 'rank']]

# Display the DataFrame
print("\nTop 5 Unique Batters with Highest Strike Rates in Powerplay (1-6 overs) with 50 or more balls faced in group stage:")
print(top_5_powerplay_batters_df)



Top 5 Unique Batters with Highest Strike Rates in Powerplay (1-6 overs) with 50 or more balls faced in group stage:
              batter     bat_team  total_powerplay_runs  \
174  Nicholas Pooran  West Indies                    98   
269      Travis Head    Australia                   104   
93    Ibrahim Zadran  Afghanistan                    98   
56      Devon Conway  New Zealand                    66   
178   Nikolaas Davin      Namibia                    58   

     strike_rate_powerplay  rank  
174             178.181818     1  
269             160.000000     2  
93              136.111111     3  
56              113.793103     4  
178             103.571429     5  


<ipython-input-39-cc3faccf8316>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_5_powerplay_batters['rank'] = top_5_powerplay_batters['strike_rate_powerplay'].rank(method='min', ascending=False).astype(int)


In [ ]:
#HIGHEST SR MO
# Filter data for group stage and middle overs (7-15)
group_stage_middle_overs_data = data[(data['Stage'] == 'Group Stage') & (data['over_number'] >= 7) & (data['over_number'] <= 15)]

# Remove duplicate over_number values for each batter within each match in middle overs
unique_middle_overs_batters = group_stage_middle_overs_data[['match_id', 'batter', 'bat_team']].drop_duplicates()

# Group by batter to calculate total runs scored by each batter in middle overs
batter_total_middle_overs_runs = group_stage_middle_overs_data.groupby('batter')['total_runs'].sum().reset_index(name='total_middle_overs_runs')

# Calculate balls faced by each batter in middle overs
batter_balls_faced_middle_overs = group_stage_middle_overs_data.groupby('batter')['overs_actual'].count().reset_index(name='balls_faced_middle_overs')

# Merge total runs and balls faced data
batter_middle_overs_stats = pd.merge(batter_total_middle_overs_runs, batter_balls_faced_middle_overs, on='batter')

# Calculate strike rate for each batter in middle overs
batter_middle_overs_stats['strike_rate_middle_overs'] = (batter_middle_overs_stats['total_middle_overs_runs'] / batter_middle_overs_stats['balls_faced_middle_overs']) * 100

# Merge with 'bat_team' to get team information
batter_middle_overs_stats = pd.merge(batter_middle_overs_stats, unique_middle_overs_batters[['batter', 'bat_team']], on='batter')

# Filter batters with 50 or more balls faced in middle overs
batter_middle_overs_stats = batter_middle_overs_stats[batter_middle_overs_stats['balls_faced_middle_overs'] >= 50]

# Sort by strike rate in descending order and drop duplicates to keep unique batters
unique_batters_sorted = batter_middle_overs_stats.sort_values(by='strike_rate_middle_overs', ascending=False).drop_duplicates(subset=['batter'])

# Get top 5 unique batters with highest strike rates in middle overs
top_5_middle_overs_batters = unique_batters_sorted.head(5)

# Add rank column
top_5_middle_overs_batters['rank'] = top_5_middle_overs_batters['strike_rate_middle_overs'].rank(method='min', ascending=False).astype(int)

# Save the top 5 batters with highest strike rates in middle overs to a DataFrame
top_5_middle_overs_batters_df = top_5_middle_overs_batters[['batter', 'bat_team', 'total_middle_overs_runs', 'strike_rate_middle_overs', 'rank']]

# Display the DataFrame
print("\nTop 5 Unique Batters with Highest Strike Rates in Middle Overs (7-15 overs) with 50 or more balls faced in group stage:")
print(top_5_middle_overs_batters_df)



Top 5 Unique Batters with Highest Strike Rates in Middle Overs (7-15 overs) with 50 or more balls faced in group stage:
                 batter     bat_team  total_middle_overs_runs  \
44     Brandon McMullen     Scotland                      101   
182      Marcus Stoinis    Australia                       92   
238  Rahmanullah Gurbaz  Afghanistan                       82   
123      Ibrahim Zadran  Afghanistan                       68   
307       Towhid Hridoy   Bangladesh                       82   

     strike_rate_middle_overs  rank  
44                 202.000000     1  
182                184.000000     2  
238                143.859649     3  
123                128.301887     4  
307                128.125000     5  


<ipython-input-40-acc17f749939>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_5_middle_overs_batters['rank'] = top_5_middle_overs_batters['strike_rate_middle_overs'].rank(method='min', ascending=False).astype(int)


In [ ]:
#HIGHEST SR MO

# Filter data for group stage and death overs (16-20)
group_stage_death_overs_data = data[(data['Stage'] == 'Group Stage') & (data['over_number'] >= 16) & (data['over_number'] <= 20)]

# Remove duplicate over_number values for each batter within each match in death overs
unique_death_overs_batters = group_stage_death_overs_data[['match_id', 'batter', 'bat_team']].drop_duplicates()

# Group by batter to calculate total runs scored by each batter in death overs
batter_total_death_overs_runs = group_stage_death_overs_data.groupby('batter')['total_runs'].sum().reset_index(name='total_death_overs_runs')

# Calculate balls faced by each batter in death overs
batter_balls_faced_death_overs = group_stage_death_overs_data.groupby('batter')['overs_actual'].count().reset_index(name='balls_faced_death_overs')

# Merge total runs and balls faced data
batter_death_overs_stats = pd.merge(batter_total_death_overs_runs, batter_balls_faced_death_overs, on='batter')

# Calculate strike rate for each batter in death overs
batter_death_overs_stats['strike_rate_death_overs'] = (batter_death_overs_stats['total_death_overs_runs'] / batter_death_overs_stats['balls_faced_death_overs']) * 100

# Merge with 'bat_team' to get team information
batter_death_overs_stats = pd.merge(batter_death_overs_stats, unique_death_overs_batters[['batter', 'bat_team']], on='batter')

# Filter batters with 20 or more balls faced in death overs
batter_death_overs_stats = batter_death_overs_stats[batter_death_overs_stats['balls_faced_death_overs'] >= 20]

# Sort by strike rate in descending order and drop duplicates to keep unique batters
unique_batters_sorted = batter_death_overs_stats.sort_values(by='strike_rate_death_overs', ascending=False).drop_duplicates(subset=['batter'])

# Get top 5 unique batters with highest strike rates in death overs
top_5_death_overs_batters = unique_batters_sorted.head(5)

# Add rank column
top_5_death_overs_batters['rank'] = top_5_death_overs_batters['strike_rate_death_overs'].rank(method='min', ascending=False).astype(int)

# Save the top 5 batters with highest strike rates in death overs to a DataFrame
top_5_death_overs_batters_df = top_5_death_overs_batters[['batter', 'bat_team', 'total_death_overs_runs', 'strike_rate_death_overs', 'rank']]

# Display the DataFrame
print("\nTop 5 Unique Batters with Highest Strike Rates in Death Overs (16-20 overs) with 20 or more balls faced in group stage:")
print(top_5_death_overs_batters_df)



Top 5 Unique Batters with Highest Strike Rates in Death Overs (16-20 overs) with 20 or more balls faced in group stage:
                  batter                  bat_team  total_death_overs_runs  \
132       Marcus Stoinis                 Australia                      67   
211  Sherfane Rutherford               West Indies                      57   
1            Aaron Jones  United States of America                      56   
225            Tim David                 Australia                      49   
178    Richie Berrington                  Scotland                      46   

     strike_rate_death_overs  rank  
132               203.030303     1  
211               196.551724     2  
1                 193.103448     3  
225               181.481481     4  
178               176.923077     5  


<ipython-input-42-cf878d72ca0c>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_5_death_overs_batters['rank'] = top_5_death_overs_batters['strike_rate_death_overs'].rank(method='min', ascending=False).astype(int)


In [ ]:
#pp wickets

# Step 1: Filter rows for power play overs (over_number between 1 and 6)
power_play_data = data[(data['over_number'] >= 1) & (data['over_number'] <= 6)]

# Step 2: Filter rows where wickets were taken (out_player_id is not null) within power play overs and not run out
power_play_wickets_data = power_play_data[(power_play_data['out_player_id'].notnull()) & (power_play_data['dismissal_text_shot'] != 'run out')]

# Step 3: Group by bowler_player_id and count occurrences of out_player_id
power_play_wickets_count = power_play_wickets_data.groupby('bowler_player_id')['out_player_id'].count()

# Step 4: Sort the wickets_count in descending order and select top 3
top_3_power_play_wicket_takers = power_play_wickets_count.nlargest(3)

# Step 5: Retrieve names and bowl teams for the top 3 wicket takers
top_3_power_play_info = (
    data[data['bowler_player_id'].isin(top_3_power_play_wicket_takers.index)]
    .drop_duplicates(subset=['bowler_player_id'])  # Ensure unique bowlers
    [['bowler_player_id', 'bowler', 'bowl_team']]
    .merge(top_3_power_play_wicket_takers.rename('total_wickets_PP'), left_on='bowler_player_id', right_index=True)
    .sort_values(by='total_wickets_PP', ascending=False)
    .reset_index(drop=True)
)

# Step 6: Add rank column
top_3_power_play_info['rank'] = top_3_power_play_info['total_wickets_PP'].rank(method='min', ascending=False).astype(int)

# Display the sorted top 3 wicket takers in power play along with their names, bowl teams, total wickets, and rank
print("Top 3 Wicket Takers in Power Play:")
print(top_3_power_play_info[['bowler', 'bowl_team', 'total_wickets_PP', 'rank']])


Top 3 Wicket Takers in Power Play:
             bowler     bowl_team  total_wickets_PP  rank
0  Fazalhaq Farooqi   Afghanistan                10     1
1     Naveen-ul-Haq   Afghanistan                 8     2
2     Kagiso Rabada  South Africa                 7     3


In [ ]:
#HIGHEST WK TAKER IN Middle overs

# Step 1: Filter rows for middle overs (over_number between 7 and 15)
middle_overs_data = data[(data['over_number'] >= 7) & (data['over_number'] <= 15)]

# Step 2: Filter rows where wickets were taken (out_player_id is not null) within middle overs
middle_overs_wickets_data = middle_overs_data[middle_overs_data['out_player_id'].notnull() & (middle_overs_data['dismissal_text_shot'] != 'run out')]

# Step 3: Group by bowler_player_id and count occurrences of out_player_id
middle_overs_wickets_count = middle_overs_wickets_data.groupby('bowler_player_id')['out_player_id'].count()

# Step 4: Sort the wickets_count in descending order and select top 5
top_5_middle_overs_wicket_takers = middle_overs_wickets_count.nlargest(5)

# Step 5: Retrieve names and bowl teams for the top 8 wicket takers
top_5_middle_overs_info = (
    data[data['bowler_player_id'].isin(top_5_middle_overs_wicket_takers.index)]
    .drop_duplicates(subset=['bowler_player_id'])  # Ensure unique bowlers
    [['bowler_player_id', 'bowler', 'bowl_team']]
    .merge(top_5_middle_overs_wicket_takers.rename('total_wickets_MO'), left_on='bowler_player_id', right_index=True)
    .sort_values(by='total_wickets_MO', ascending=False)
    .reset_index(drop=True)
)

# Step 6: Add rank column
top_5_middle_overs_info['rank'] = top_5_middle_overs_info['total_wickets_MO'].rank(method='min', ascending=False).astype(int)

# Display the sorted top 8 wicket takers in middle overs along with their names, bowl teams, total wickets, and rank
print("Top 5 Wicket Takers in Middle Overs:")
print(top_5_middle_overs_info[['bowler', 'bowl_team', 'total_wickets_MO', 'rank']])


Top 5 Wicket Takers in Middle Overs:
          bowler     bowl_team  total_wickets_MO  rank
0    Rashid Khan   Afghanistan                11     1
1  Anrich Nortje  South Africa                10     2
2     Adam Zampa     Australia                10     2
3    Adil Rashid       England                 9     4
4  Kuldeep Yadav         India                 9     4


In [ ]:
#HIGHEST WK TAKER IN Death overs

# Step 1: Filter rows for death overs (over_number greater than 15)
death_overs_data = data[data['over_number'] > 15]

# Step 2: Filter rows where wickets were taken (out_player_id is not null) within death overs
death_overs_wickets_data = death_overs_data[death_overs_data['out_player_id'].notnull() & (death_overs_data['dismissal_text_shot'] != 'run out')]

# Step 3: Group by bowler_player_id and count occurrences of out_player_id
death_overs_wickets_count = death_overs_wickets_data.groupby('bowler_player_id')['out_player_id'].count()

# Step 4: Sort the wickets_count in descending order and select top
top_5_death_overs_wicket_takers = death_overs_wickets_count.nlargest(5)

# Step 5: Retrieve names and bowl teams for the top 8 wicket takers
top_5_death_overs_info = (
    data[data['bowler_player_id'].isin(top_5_death_overs_wicket_takers.index)]
    .drop_duplicates(subset=['bowler_player_id'])  # Ensure unique bowlers
    [['bowler_player_id', 'bowler', 'bowl_team']]
    .merge(top_5_death_overs_wicket_takers.rename('total_wickets_death'), left_on='bowler_player_id', right_index=True)
    .sort_values(by='total_wickets_death', ascending=False)
    .reset_index(drop=True)
)

# Step 6: Add rank column
top_5_death_overs_info['rank'] = top_5_death_overs_info['total_wickets_death'].rank(method='min', ascending=False).astype(int)

# Display the sorted top 8 wicket takers in death overs along with their names, bowl teams, total wickets, and rank
print("Top 5 Wicket Takers in Death Overs:")
print(top_5_death_overs_info[['bowler', 'bowl_team', 'total_wickets_death', 'rank']])




Top 5 Wicket Takers in Death Overs:
           bowler  bowl_team  total_wickets_death  rank
0  Arshdeep Singh      India                    9     1
1    Chris Jordan    England                    9     1
2     Pat Cummins  Australia                    8     3
3  Jasprit Bumrah      India                    7     4
4  Marcus Stoinis  Australia                    6     5


In [ ]:
#Top 5 Bowlers with Lowest Economy Rates in Powerplay (8 or More Overs Bowled):

# Filter data for powerplay overs (1-6)
powerplay_data = data[(data['over_number'] >= 1) & (data['over_number'] <= 6)]

# Remove duplicate over_number values for each bowler within each match in powerplay
unique_powerplay_overs = powerplay_data[['match_id', 'bowler', 'bowl_team', 'over_number']].drop_duplicates()

# Group by bowler to count the unique overs bowled by each bowler in powerplay
bowler_total_powerplay_overs = unique_powerplay_overs.groupby('bowler')['over_number'].count().reset_index(name='total_powerplay_overs')

# Group by bowler to sum the total runs conceded by each bowler in powerplay
bowler_total_powerplay_runs_conceded = powerplay_data.groupby('bowler')['total_runs'].sum().reset_index(name='total_powerplay_runs_conceded')

# Calculate the number of wickets taken by each bowler in powerplay
powerplay_wickets_data = powerplay_data[powerplay_data['out_player_id'].notnull() & (power_play_data['dismissal_text_shot'] != 'run out')]
powerplay_wickets_count = powerplay_wickets_data.groupby('bowler')['out_player_id'].count().reset_index(name='powerplay_wickets_count')

# Calculate the economy rate for each bowler in powerplay
bowler_powerplay_economy_rate = pd.merge(bowler_total_powerplay_overs, bowler_total_powerplay_runs_conceded, on='bowler')
bowler_powerplay_economy_rate['powerplay_economy_rate'] = bowler_powerplay_economy_rate['total_powerplay_runs_conceded'] / bowler_powerplay_economy_rate['total_powerplay_overs']

# Merge powerplay wickets count into bowler_powerplay_economy_rate
bowler_powerplay_economy_rate = pd.merge(bowler_powerplay_economy_rate, powerplay_wickets_count, on='bowler')

# Merge bowl_team into bowler_powerplay_economy_rate
bowler_powerplay_economy_rate = pd.merge(bowler_powerplay_economy_rate, data[['bowler', 'bowl_team']].drop_duplicates(), on='bowler', how='left')

# Filter bowlers who have bowled 10 or more overs in powerplay and taken at least 1 wicket
qualified_powerplay_bowlers = bowler_powerplay_economy_rate[(bowler_powerplay_economy_rate['total_powerplay_overs'] >= 10) & (bowler_powerplay_economy_rate['powerplay_wickets_count'] >= 1)]

# Get top 5 bowlers with lowest economy rates in powerplay
top_5_powerplay_bowlers_eco = qualified_powerplay_bowlers.nsmallest(5, 'powerplay_economy_rate')

# Display the top 5 bowlers with lowest economy rates in powerplay
print("\nTop 5 Bowlers with Lowest Economy Rates in Powerplay (10 or More Overs Bowled):")
print(top_5_powerplay_bowlers_eco[['bowler', 'bowl_team', 'powerplay_economy_rate', 'powerplay_wickets_count']])



Top 5 Bowlers with Lowest Economy Rates in Powerplay (10 or More Overs Bowled):
                bowler     bowl_team  powerplay_economy_rate  \
29      Jasprit Bumrah         India                4.285714   
35       Kagiso Rabada  South Africa                5.000000   
32      Josh Hazlewood     Australia                5.071429   
54       Naveen-ul-Haq   Afghanistan                5.200000   
74  Tanzim Hasan Sakib    Bangladesh                5.214286   

    powerplay_wickets_count  
29                        5  
35                        7  
32                        3  
54                        8  
74                        7  


In [ ]:
#TOP 3 BEST ECO IN MIDDLE (18 or more overs bowled)


# Filter data for middle overs (7th to 15th overs)
middle_overs_data = data[(data['over_number'] >= 7) & (data['over_number'] <= 15)]

# Remove duplicate over_number values for each bowler within each match in middle overs
unique_middle_overs = middle_overs_data[['match_id', 'bowler', 'bowl_team', 'over_number']].drop_duplicates()

# Group by bowler to count the unique overs bowled by each bowler in middle overs
bowler_total_middle_overs = unique_middle_overs.groupby('bowler')['over_number'].count().reset_index(name='total_middle_overs')

# Group by bowler to sum the total runs conceded by each bowler in middle overs
bowler_total_middle_runs_conceded = middle_overs_data.groupby('bowler')['total_runs'].sum().reset_index(name='total_middle_runs_conceded')

# Calculate the number of wickets taken by each bowler in middle overs
middle_wickets_data = middle_overs_data[middle_overs_data['out_player_id'].notnull() & (middle_overs_data['dismissal_text_shot'] != 'run out')]
middle_wickets_count = middle_wickets_data.groupby('bowler')['out_player_id'].count().reset_index(name='middle_wickets_count')

# Calculate the economy rate for each bowler in middle overs
bowler_middle_economy_rate = pd.merge(bowler_total_middle_overs, bowler_total_middle_runs_conceded, on='bowler')
bowler_middle_economy_rate['middle_economy_rate'] = bowler_middle_economy_rate['total_middle_runs_conceded'] / bowler_middle_economy_rate['total_middle_overs']

# Merge middle wickets count into bowler_middle_economy_rate
bowler_middle_economy_rate = pd.merge(bowler_middle_economy_rate, middle_wickets_count, on='bowler')

# Merge bowl_team into bowler_middle_economy_rate
bowler_middle_economy_rate = pd.merge(bowler_middle_economy_rate, data[['bowler', 'bowl_team']].drop_duplicates(), on='bowler', how='left')

# Filter bowlers who have bowled 8 or more overs in middle overs and taken at least 1 wicket
qualified_middle_bowlers = bowler_middle_economy_rate[(bowler_middle_economy_rate['total_middle_overs'] >= 21) & (bowler_middle_economy_rate['middle_wickets_count'] >= 1)]

# Get top 5 bowlers with lowest economy rates in middle overs
top_5_middle_bowlers_eco = qualified_middle_bowlers.nsmallest(5, 'middle_economy_rate')

# Display the top 5 bowlers with lowest economy rates in middle overs
print("\nTop 5 Bowlers with Lowest Economy Rates in Middle Overs (7th to 15th overs):")
print(top_5_middle_bowlers_eco[['bowler', 'bowl_team', 'middle_economy_rate', 'middle_wickets_count']])



Top 5 Bowlers with Lowest Economy Rates in Middle Overs (7th to 15th overs):
            bowler     bowl_team  middle_economy_rate  middle_wickets_count
7    Anrich Nortje  South Africa             5.291667                    10
77     Rashid Khan   Afghanistan             5.913043                    11
47  Keshav Maharaj  South Africa             5.954545                     7
1      Adil Rashid       England             6.000000                     9
0       Adam Zampa     Australia             6.347826                    10


In [ ]:
# Filter data for death overs (16th over and beyond)
death_overs_data = data[data['over_number'] >= 16]

# Remove duplicate over_number values for each bowler within each match in death overs
unique_death_overs = death_overs_data[['match_id', 'bowler', 'bowl_team', 'over_number']].drop_duplicates()

# Group by bowler to count the unique overs bowled by each bowler in death overs
bowler_total_death_overs = unique_death_overs.groupby('bowler')['over_number'].count().reset_index(name='total_death_overs')

# Group by bowler to sum the total runs conceded by each bowler in death overs
bowler_total_death_runs_conceded = death_overs_data.groupby('bowler')['total_runs'].sum().reset_index(name='total_death_runs_conceded')

# Calculate the number of wickets taken by each bowler in death overs
death_wickets_data = death_overs_data[death_overs_data['out_player_id'].notnull() & (death_overs_data['dismissal_text_shot'] != 'run out')]
death_wickets_count = death_wickets_data.groupby('bowler')['out_player_id'].count().reset_index(name='death_wickets_count')

# Calculate the economy rate for each bowler in death overs
bowler_death_economy_rate = pd.merge(bowler_total_death_overs, bowler_total_death_runs_conceded, on='bowler')
bowler_death_economy_rate['death_economy_rate'] = bowler_death_economy_rate['total_death_runs_conceded'] / bowler_death_economy_rate['total_death_overs']

# Merge death wickets count into bowler_death_economy_rate
bowler_death_economy_rate = pd.merge(bowler_death_economy_rate, death_wickets_count, on='bowler')

# Merge bowl_team into bowler_death_economy_rate
bowler_death_economy_rate = pd.merge(bowler_death_economy_rate, data[['bowler', 'bowl_team']].drop_duplicates(), on='bowler', how='left')

# Filter bowlers who have bowled 18 or more overs in death overs and taken at least 1 wicket
qualified_death_bowlers = bowler_death_economy_rate[(bowler_death_economy_rate['total_death_overs'] >= 8) & (bowler_death_economy_rate['death_wickets_count'] >= 1)]

# Get top 3 bowlers with lowest economy rates in death overs
top_3_death_bowlers_eco = qualified_death_bowlers.nsmallest(3, 'death_economy_rate')

# Add rank column
top_3_death_bowlers_eco['rank'] = top_3_death_bowlers_eco['death_economy_rate'].rank(method='min', ascending=True).astype(int)

# Display the top 3 bowlers with lowest economy rates in death overs
print("\nTop 3 Bowlers with Lowest Economy Rates in Death Overs (16th over and beyond):")
print(top_3_death_bowlers_eco[['bowler', 'bowl_team', 'death_economy_rate', 'death_wickets_count', 'rank']])



Top 3 Bowlers with Lowest Economy Rates in Death Overs (16th over and beyond):
               bowler     bowl_team  death_economy_rate  death_wickets_count  \
48  Mustafizur Rahman    Bangladesh            4.222222                    5   
29     Jasprit Bumrah         India            4.833333                    7   
33      Kagiso Rabada  South Africa            6.777778                    5   

    rank  
48     1  
29     2  
33     3  


In [ ]:
# Concatenate all DataFrames into one DataFrame
combined_df_concat = pd.concat([top_5_run_scorers, top_5_fours, top_5_six, top_5_wkts_info, top_5_bowlers_eco, wides_stats_by_team, dots_by_team, most_dot_balls_batters, boundary_rate_df, boundary_rate_batter_df, top_5_bowlers_boundary_frequency, result_df, result_df_mo, result_df_do, top_5_powerplay_batters_df, top_5_middle_overs_batters_df, top_5_death_overs_batters_df , top_3_power_play_info, top_5_middle_overs_info, top_5_death_overs_info, top_5_powerplay_bowlers_eco, top_5_middle_bowlers_eco, top_3_death_bowlers_eco], ignore_index=True)

# Save the concatenated DataFrame to a CSV file
combined_df_concat.to_excel('AFG_results_concat.xlsx', index=False)

print("AFG_results_concat.xlsx")

